In [35]:
#Import necessary libraries
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, MaxPooling2D, Dropout, Cropping2D
from keras.layers.convolutional import Conv2D
from keras.callbacks import ModelCheckpoint

In [36]:
Path1 = '/home/carnd/CarND-Behavioral-Cloning-P3/data/data/'
#Path2 = '/home/carnd/CarND-Behavioral-Cloning-P3/data/data/'
csvFile = Path1 + 'driving_log.csv'
imgPath = Path1 + 'IMG/'

#The first row is the heading. So i am skipping it
Imgsamples = []
with open(csvFile) as file:
    reader = csv.reader(file)
    next(reader)  
    for line in reader:
        Imgsamples.append(line)

In [38]:
#Split the train and test samples. 20% for test images
trainSamples, validationSamples = train_test_split(Imgsamples, test_size=0.2) 

In [39]:
steerCorrection = 0.35 #Steering correction for left and right camera images

In [40]:
#Generator data 
colorSpace =  cv2.COLOR_BGR2RGB #since cv2 uses BGR format. Got to know this from later projects ;-)
def generator_data(samples, batch_size=32):
    num_samples = len(samples) #Length of samples
    while 1: # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            Steerangles = []
            for batch_sample in batch_samples:

                # center camera images and steering angle
                name = imgPath+batch_sample[0].split('/')[-1]
                center_image = cv2.cvtColor(cv2.imread(name), colorSpace)
                center_angle = float(batch_sample[3])

                # left camera images and steering angle
                name = imgPath+batch_sample[1].split('/')[-1]
                left_image = cv2.cvtColor(cv2.imread(name), colorSpace)
                left_angle = center_angle + steerCorrection
                left_angle = min(max(left_angle, -1.0), 1.0)

                # right camera images and steering angle
                name = imgPath+batch_sample[2].split('/')[-1]
                right_image = cv2.cvtColor(cv2.imread(name), colorSpace)
                right_angle = center_angle - steerCorrection
                right_angle = min(max(right_angle, -1.0), 1.0)

                
                images.append(center_image)
                Steerangles.append(center_angle)
                images.append(left_image)
                Steerangles.append(left_angle)
                images.append(right_image)
                Steerangles.append(right_angle)
            #Collect all the camera images    
            X_train_orig = np.array(images)
            y_train_orig = np.array(Steerangles)
            
            # Flip the images
            X_train_flip = np.array(np.fliplr(images))
            y_train_flip =-np.array(Steerangles)
            
            #Concatenate the images
            X_train = np.concatenate((X_train_orig, X_train_flip), axis=0)
            y_train = np.concatenate((y_train_orig, y_train_flip), axis=0)
            
            yield sklearn.utils.shuffle(X_train, y_train)

In [41]:
#Data generator use
train_generator = generator_data(trainSamples, batch_size=32)
validation_generator = generator_data(validationSamples, batch_size=32)

In [42]:
IMG1 = cv2.cvtColor(cv2.imread(imgPath+trainSamples[0][0].split('/')[-1]), colorSpace)
print(IMG1.shape)
shape = IMG1.shape

(160, 320, 3)


In [43]:
'''
model = Sequential()
model.add(Lambda(lambda x: ((x/127.0)-0.5),input_shape=shape,name='lambda'))
model.add(Cropping2D(cropping=((60,30),(0,0)),name='crop'))
model.add(Conv2D(nb_filter=24,nb_row=5,nb_col=5,subsample=(1,2),border_mode='valid',activation='elu',name='conv1'))
model.add(Conv2D(nb_filter=36,nb_row=5,nb_col=5,subsample=(2,2),border_mode='valid',activation='elu',name='conv2'))
model.add(Conv2D(nb_filter=48,nb_row=5,nb_col=5,subsample=(2,2),border_mode='valid',activation='elu',name='conv3'))
model.add(Conv2D(nb_filter=64,nb_row=3,nb_col=3,subsample=(2,2),border_mode='valid',activation='elu',name='conv4'))
model.add(Conv2D(nb_filter=64,nb_row=3,nb_col=3,subsample=(2,2),border_mode='valid',activation='elu',name='conv5'))
model.add(Flatten(name='flat'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(1164, activation='elu', name='dense1'))
model.add(Dense(100, activation='elu', name='dense2'))
model.add(Dense(50, activation='elu', name='dense3'))
model.add(Dense(10, activation='elu', name='dense4'))
model.add(Dense(1, name='output'))
model.compile(loss='mse',optimizer='adam')
'''

In [44]:
#Network architecture
model = Sequential()
model.add(Lambda(lambda x: ((x/255.0)-0.5),input_shape=shape,name='lambda'))
model.add(Cropping2D(cropping=((50,20),(0,0)),name='crop'))
model.add(Conv2D(nb_filter=24,nb_row=5,nb_col=5,subsample=(2,2),border_mode='valid',activation='elu',name='conv1'))
model.add(Conv2D(nb_filter=36,nb_row=5,nb_col=5,subsample=(2,2),border_mode='valid',activation='elu',name='conv2'))
model.add(Conv2D(nb_filter=48,nb_row=5,nb_col=5,subsample=(2,2),border_mode='valid',activation='elu',name='conv3'))
model.add(Conv2D(nb_filter=64,nb_row=3,nb_col=3,subsample=(2,2),border_mode='valid',activation='elu',name='conv4'))
model.add(Conv2D(nb_filter=64,nb_row=3,nb_col=3,subsample=(2,2),border_mode='valid',activation='elu',name='conv5'))
model.add(Flatten(name='flat'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(100, activation='elu', name='denseOne'))
model.add(Dense(50, activation='elu', name='denseTwo'))
model.add(Dense(10, activation='elu', name='denseThree'))
model.add(Dense(1, name='output'))
model.compile(loss='mse',optimizer='adam')


"\nmodel = Sequential()\nmodel.add(Lambda(lambda x: ((x/255.0)-0.5),\n                 input_shape=shape,\n                 name='lambda'))\nmodel.add(Cropping2D(cropping=((50,20),(0,0)),\n                     name='crop'))\nmodel.add(Conv2D(nb_filter=24,\n                 nb_row=5,\n                 nb_col=5,\n                 subsample=(2,2),\n                 border_mode='valid',\n                 activation='elu',\n                 name='conv1'))\nmodel.add(Conv2D(nb_filter=36,\n                 nb_row=5,\n                 nb_col=5,\n                 subsample=(2,2),\n                 border_mode='valid',\n                 activation='elu',\n                 name='conv2'))\nmodel.add(Conv2D(nb_filter=48,\n                 nb_row=5,\n                 nb_col=5,\n                 subsample=(2,2),\n                 border_mode='valid',\n                 activation='elu',\n                 name='conv3'))\nmodel.add(Conv2D(nb_filter=64,\n                 nb_row=3,\n                 nb_co

In [45]:
save_filename = 'model.h5'
'''checkpoint = ModelCheckpoint(save_filename,monitor='val_loss',save_best_only=True,save_weights_only=False)
callbacks_list = [checkpoint]'''


"checkpoint = ModelCheckpoint(save_filename,\n                             monitor='val_loss',\n                             save_best_only=True,\n                             save_weights_only=False)\ncallbacks_list = [checkpoint]"

In [46]:
history_obj =  model.fit_generator(train_generator,
                                   samples_per_epoch=len(trainSamples)*6,
                                   validation_data=validation_generator,
                                   nb_val_samples=len(validationSamples)*6,
                                   nb_epoch=6)
model.save(save_filename)

Epoch 1/6
38568/38568 [==============================] - 116s - loss: 0.1396 - val_loss: 0.0586
Epoch 2/6
38568/38568 [==============================] - 116s - loss: 0.0554 - val_loss: 0.0500
Epoch 3/6
38568/38568 [==============================] - 116s - loss: 0.0485 - val_loss: 0.0454
Epoch 4/6
38568/38568 [==============================] - 116s - loss: 0.0445 - val_loss: 0.0421
Epoch 5/6
38568/38568 [==============================] - 116s - loss: 0.0417 - val_loss: 0.0396
Epoch 6/6
38568/38568 [==============================] - 116s - loss: 0.0399 - val_loss: 0.0376
